In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import os
import seaborn as sns
from scipy import stats
import locale
locale.setlocale(locale.LC_ALL, '')

from sklearn import model_selection
from sklearn import preprocessing
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


#Functions we created:
from modules import dem_fx
from modules import transaction_fx as trns
from modules import plot_functions as plt_fx
from modules import ml_functions as ml_fx

os.getcwd()

#Allows reload of modules:
%load_ext autoreload
%autoreload

In [ ]:
weekly_cart_df_filtered_labels = pd.read_csv("saved_structures/weekly_cart_df_filtered_labels.csv", sep = '\t')

In [ ]:
weekly_cart_df_filtered_labels.head()

In [ ]:
products = pd.read_csv("saved_structures/updated_prod.csv", sep = "\t")

In [ ]:
products.head()

In [ ]:
hh_demographic = pd.read_csv(
    '../data/dunnhumby_complete_csv/hh_demographic.csv', sep=',')

transaction_data = pd.read_csv(
    '../data/dunnhumby_complete_csv/transaction_data.csv', sep=',')


In [ ]:
transaction_data.head()

In [ ]:
time = transaction_data.TRANS_TIME.unique()
time = time.tolist()
time.sort()
print(time)

In [ ]:
temp = transaction_data["TRANS_TIME"].astype("str")
for i in range(len(temp)):
    if i%100000 == 0:
        print(i)
    if len(temp[i]) == 3:
        temp[i] = "0" + temp[i]
    elif len(temp[i]) == 2:
        temp[i] = "00" + temp[i]
    elif len(temp[i]) == 1:
        temp[i] = "000" + temp[i]


In [ ]:
transaction_data["TRANS_TIME_STR"] = temp

In [ ]:
time = pd.to_datetime(transaction_data["TRANS_TIME_STR"], format='%H%M').dt.time

In [ ]:
print(time)

In [ ]:
transaction_data["transaction_time"] = time

In [ ]:
transaction_data.head()

In [ ]:
transaction_data["transaction_time"].hist(bins = 24)